In [1]:
import json
import re
from konlpy.tag import Okt
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
song=pd.read_csv("song_tagged_2.csv",lineterminator='\n',index_col=0)

In [3]:
song

,index,Genre,Artist,Title,Lyric,Date,Emotion,Proba
0,0,"발라드, 국내드라마",멜로망스,사랑인가 봐,너와 함께 하고 싶은 일들을상상하는 게요즘 내 일상이 되고너의 즐거워하는 모습을 보...,2022.02.18,슬픔,64.12
1,1,발라드,경서,나의 X에게,우리 다시 만날래예쁘게 빛나던모든 추억들이너무 그리워너의 품에 안길래이 밤이 새도록...,2022.04.24,슬픔,55.27
2,2,발라드,김민석 (멜로망스),취중고백,뭐하고 있었니 늦었지만잠시 나올래너의 집 골목에 있는놀이터에 앉아 있어친구들 만나서...,2021.12.19,슬픔,70.52
3,3,"발라드, 국내드라마",임영웅,사랑은 늘 도망가,눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨...,2021.10.11,슬픔,79.44
4,4,발라드,MSG워너비(M.O.M),듣고 싶을까,우리 함께 듣던 그 노랠 듣고 싶을까듣고 나서 잠시 날 생각은 할까아주 혹시라도 넌...,2022.02.26,슬픔,68.81
...,...,...,...,...,...,...,...,...
3569,3995,성인가요/트로트,조항조,만약에,만약에 당신이 그 누구와 사랑에 빠지면 그 사랑을 위해서 무얼할 수 있나 텅빈세상 ...,2020.07.31,슬픔,62.06
3570,3996,성인가요/트로트,송가인,용두산 엘레지,용두산아 용두산아너만은 변치 말자한 발 올려 맹세하고두 발 디뎌 언약하던한 계단 두...,2021.03.21,슬픔,44.35
3571,3997,성인가요/트로트,송가인,단장의 미아리 고개,미아리 눈물고개임이 넘던 이별 고개화약 연기 앞을 가려눈 못 뜨고 헤매일 때당신은 ...,2020.12.26,슬픔,48.68
3572,3998,성인가요/트로트,조항조,때,낮춰야 할 때는 낮춰야하고 물러나야 할 때는 물러나야지 오르고 오르면 끝이 있듯이 ...,2016.07.13,슬픔,66.13


In [23]:
song["Genre"].value_counts()

댄스                        487
랩/힙합                      451
성인가요/트로트                  439
R&B;/Soul                 418
발라드                       377
록/메탈                      332
인디음악, 포크/블루스              254
포크/블루스                    149
발라드, 인디음악                 120
인디음악, 록/메탈                119
발라드, 국내드라마                 79
R&B;/Soul, 인디음악            76
랩/힙합, 인디음악                 74
록/메탈, 국내드라마                46
포크/블루스, 국내드라마              43
R&B;/Soul, 국내드라마           16
댄스, 랩/힙합                   14
인디음악, 일렉트로니카               12
성인가요/트로트, 국내드라마             9
발라드, R&B;/Soul              8
발라드, 록/메탈                   6
랩/힙합, 국내드라마                 5
발라드, R&B;/Soul, 국내드라마       5
발라드, 랩/힙합, 국내드라마            4
발라드, R&B/Soul               4
댄스, 인디음악                    3
발라드, 포크/블루스                 3
록/메탈, 애니메이션/웹툰              3
발라드, 게임                     2
발라드, 랩/힙합                   2
발라드, 국내영화                   2
록/메탈, 국내영화                  2
성인가요/트로트, 국내영화              1
R&B;/Soul,

### Countervectorizer를 이용하여 장르 유사도 구하기

In [13]:
from sklearn.feature_extraction.text import CountVectorizer  # 피체 벡터화
from sklearn.metrics.pairwise import cosine_similarity  # 코사인 유사도


count_vect_category = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_category = count_vect_category.fit_transform(song["Genre"]) 
song_simi_genre = cosine_similarity(genre_category, genre_category) 
#place_simi_cate_sorted_ind = place_simi_cate.argsort()[:, ::-1]

In [14]:
cosine_similarity(genre_category, genre_category).mean()

0.13293403805434764

In [15]:
genre_result=pd.DataFrame(song_simi_genre)
genre_result.columns = song["Genre"]
genre_result.index = song["Genre"]
genre_result

Genre,"발라드, 국내드라마",발라드,발라드,"발라드, 국내드라마",발라드,발라드,발라드,발라드,발라드,발라드,...,성인가요/트로트,성인가요/트로트,성인가요/트로트,성인가요/트로트,성인가요/트로트,성인가요/트로트,성인가요/트로트,성인가요/트로트,성인가요/트로트,성인가요/트로트
Genre,,,,,,,,,,,,,,,,,,,,,
"발라드, 국내드라마",1.00000,0.57735,0.57735,1.00000,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
발라드,0.57735,1.00000,1.00000,0.57735,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
발라드,0.57735,1.00000,1.00000,0.57735,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"발라드, 국내드라마",1.00000,0.57735,0.57735,1.00000,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
발라드,0.57735,1.00000,1.00000,0.57735,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
성인가요/트로트,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
성인가요/트로트,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
성인가요/트로트,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [38]:
genre_result.iloc[:,0].sort_values(ascending=False).head(20)

Genre
발라드, 국내드라마    1.0
발라드, 국내드라마    1.0
발라드, 국내드라마    1.0
발라드, 국내드라마    1.0
발라드, 국내드라마    1.0
발라드, 국내드라마    1.0
발라드, 국내드라마    1.0
발라드, 국내드라마    1.0
발라드, 국내드라마    1.0
발라드, 국내드라마    1.0
발라드, 국내드라마    1.0
발라드, 국내드라마    1.0
발라드, 국내드라마    1.0
발라드, 국내드라마    1.0
발라드, 국내드라마    1.0
발라드, 국내드라마    1.0
발라드, 국내드라마    1.0
발라드, 국내드라마    1.0
발라드, 국내드라마    1.0
발라드, 국내드라마    1.0
Name: 발라드, 국내드라마, dtype: float64

### Countervectorizer를 이용하여 가수 유사도 구하기

In [40]:
song["Artist"].value_counts()

임영웅            87
송가인            66
볼빨간사춘기         55
아이유            51
10CM           43
               ..
이민혁             1
차은우 (ASTRO)     1
손담비             1
스카이             1
Nami (나미)       1
Name: Artist, Length: 947, dtype: int64

In [41]:
count_vect_artist = CountVectorizer(min_df=0, ngram_range=(1,2))
song_artist = count_vect_artist.fit_transform(song["Artist"]) 
song_simi_artist = cosine_similarity(song_artist, song_artist) 

In [42]:
song_simi_artist.mean()

0.004098369488802968

In [43]:
artist_result=pd.DataFrame(song_simi_artist)
artist_result.columns = song["Artist"]
artist_result.index =song["Artist"]
artist_result

Artist,멜로망스,경서,김민석 (멜로망스),임영웅,MSG워너비(M.O.M),경서예지,임영웅,케이시 (Kassy),임영웅,아이유,...,진해성,송가인,김소연,홍자,송가인,조항조,송가인,송가인,조항조,김의영
Artist,,,,,,,,,,,,,,,,,,,,,
멜로망스,1.00000,0.0,0.57735,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
경서,0.00000,1.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
김민석 (멜로망스),0.57735,0.0,1.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
임영웅,0.00000,0.0,0.00000,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MSG워너비(M.O.M),0.00000,0.0,0.00000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
조항조,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
송가인,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
송가인,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0


### TfidfVectorizer 이용하여 가사 유사도 구하기

In [45]:
song["Lyric"]

0       너와 함께 하고 싶은 일들을상상하는 게요즘 내 일상이 되고너의 즐거워하는 모습을 보...
1       우리 다시 만날래예쁘게 빛나던모든 추억들이너무 그리워너의 품에 안길래이 밤이 새도록...
2       뭐하고 있었니 늦었지만잠시 나올래너의 집 골목에 있는놀이터에 앉아 있어친구들 만나서...
3       눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨...
4       우리 함께 듣던 그 노랠 듣고 싶을까듣고 나서 잠시 날 생각은 할까아주 혹시라도 넌...
                              ...                        
3569    만약에 당신이 그 누구와 사랑에 빠지면 그 사랑을 위해서 무얼할 수 있나 텅빈세상 ...
3570    용두산아 용두산아너만은 변치 말자한 발 올려 맹세하고두 발 디뎌 언약하던한 계단 두...
3571    미아리 눈물고개임이 넘던 이별 고개화약 연기 앞을 가려눈 못 뜨고 헤매일 때당신은 ...
3572    낮춰야 할 때는 낮춰야하고 물러나야 할 때는 물러나야지 오르고 오르면 끝이 있듯이 ...
3573    사랑하다 헤어지고 좋아하다 미워하고 돌고도는 세상살이가 돌고도는 인생살이가 왜이리 ...
Name: Lyric, Length: 3574, dtype: object

In [46]:
vectorizer = TfidfVectorizer()
tf_lyric = vectorizer.fit_transform(song["Lyric"]).todense()
song_simi_lyric = cosine_similarity(tf_lyric, tf_lyric) 

lyric_result=pd.DataFrame(song_simi_lyric)
lyric_result.columns = song["Lyric"]
lyric_result.index = song["Lyric"]
lyric_result

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Lyric,너와 함께 하고 싶은 일들을상상하는 게요즘 내 일상이 되고너의 즐거워하는 모습을 보고 있으면자연스레 따라 웃고 있는 걸너의 행동에 설레어하고 뒤척이다가지새운 밤이 많아지는데이건 누가 봐도 사랑일 텐데종일 함께면 질릴 텐데나 돌아서도 온통 너인 건아무래도 사랑인가 봐점점 너와 하고 싶은 일들 생각하면서하룰 보낸 날이 많아지는데이건 누가 봐도 사랑일 텐데종일 함께면 질릴 텐데나 돌아서도 온통 너인 건아무래도 사랑인가 봐너의 행복해하는 모습을 보고 있으면나도 모르게 따라 웃는데이 정도면 알아줄 만하잖아너도 용기 낼만 하잖아나만 이런 게 아니라면우리 만나볼 만하잖아아무래도 사랑인가 봐,우리 다시 만날래예쁘게 빛나던모든 추억들이너무 그리워너의 품에 안길래이 밤이 새도록네게 말할 거야너를 좋아한다고안녕 그대야요즘 어떻게 지내가로수길 걷다가생각이 나서행복해 보이는 사람들 속에우리 둘만 없어서어색했었어늘 집에 갈 땐항상 뒤돌아보면날 보며 웃어주던네가 생각났어우리 다시 만날래예쁘게 빛나던모든 추억들이너무 그리워너의 품에 안길래이 밤이 새도록네게 말할 거야너를 좋아한다고토요일마다 가던한강공원에이어폰 하나씩나눠끼고서벚꽃엔딩같이 듣고 있으면부러울 게 없었어너만 있으면널 사랑할 때내가 너무 그리워날 보며 웃어주던네가 보고 싶어우리 다시 만날래예쁘게 빛나던모든 추억들이너무 그리워너의 품에 안길래이 밤이 새도록네게 말할 거야너를 좋아한다고같이 별 보러 갈래널 다시 만나면네 옆에 기대서잠들고 싶어너의 품에 안길래이 밤이 새도록내 곁에 있어줘내겐 너뿐이라고,뭐하고 있었니 늦었지만잠시 나올래너의 집 골목에 있는놀이터에 앉아 있어친구들 만나서 오랜만에술을 좀 했는데자꾸만 니 얼굴 떠올라무작정 달려왔어이 맘 모르겠니요즘 난 미친 사람처럼너만 생각해대책없이 네가 점점 좋아져아냐 안 취했어 진짜야널 정말 사랑해눈물이 날만큼 원하고 있어정말로 몰랐니가끔 전화해 장난치듯주말엔 뭐할거냐며너의 관심 끌던 나를그리고 한번씩 누나 주려 샀는데너 그냥 준다고생색 낸 선물도 너 때문에 산거야이 맘 모르겠니요즘 난 미친 사람처럼너만 생각해대책없이 네가 점점 좋아져아냐 안 취했어 진짜야널 정말 사랑해진심이야 믿어줘갑자기 이런 말 놀랐다면 미안해부담이 되는게 당연해이해해 널하지만 내 고백도 이해해 주겠니 oh지금 당장 대답하진마나와 일주일만 사귀어줄래후회없이 잘 해주고 싶은데그 후에도 니가 싫다면나 그때 포기할게귀찮게 안할게 혼자 아플게진심이야 너를 사랑하고 있어,눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨었지내 아름답던 사람아사랑이란 게 참 쓰린 거더라잡으려 할수록 더 멀어지더라이별이란 게 참 쉬운 거더라내 잊지 못할 사람아사랑아 왜 도망가수줍은 아이처럼행여 놓아버릴까 봐꼭 움켜쥐지만그리움이 쫓아 사랑은 늘 도망가잠시 쉬어가면 좋을 텐데바람이 분다 옷깃을 세워도차가운 이별의 눈물이 차올라잊지 못해서 가슴에 사무친내 소중했던 사람아사랑아 왜 도망가수줍은 아이처럼행여 놓아버릴까 봐꼭 움켜쥐지만그리움이 쫓아 사랑은 늘 도망가잠시 쉬어가면 좋을 텐데기다림도 애태움도 다 버려야 하는데무얼 찾아 이 길을 서성일까무얼 찾아 여기 있나사랑아 왜 도망가수줍은 아이처럼행여 놓아버릴까 봐꼭 움켜쥐지만그리움이 쫓아 사랑은 늘 도망가잠시 쉬어가면 좋을 텐데잠시 쉬어가면 좋을 텐데,우리 함께 듣던 그 노랠 듣고 싶을까듣고 나서 잠시 날 생각은 할까아주 혹시라도 넌 내가 보고 싶을까난 왜 니 목소리가 듣고 싶을까뻔하디 뻔한 노랫말 여서 미안해미안해도 부디 넌 꼭 들어줬음 해후회도 미련도 없는 척 떠났지만이게 내 진심인 걸 꼭 알아줬음 해우리 함께 듣던 그 노랠 듣고 싶을까듣고 나서 잠시 날 생각은 할까아주 혹시라도 넌 내가 보고 싶을까못난 내 목소리가 듣고 싶을까말없이 다 전해지는 건 없지 난 말해 지금몇 년째 나의 favorite oneYou the only one yeahYes I know 무엇도 달라질 건 없지만이게 내 진심인 걸 꼭 알아줬음 해우리 함께 듣던 그 노랠 듣고 싶을까듣고 나서 잠시 날 생각은 할까아주 혹시라도 넌 내가 보고 싶을까난 왜 니 목소리가 듣고 싶을까Are you okay not today천천히 대답해도 돼 i just stay넌 내 목소리가 듣고 싶을까누굴 만나 봐도 다시는 못할 것 같아그립고 또 그립다 우리 그 사랑WooI’m missing UNever forget UNever forget UOh no baby,끝없이 별빛이 내리던 밤기분 좋은 바람이두 빰을 스치고새벽 바다 한곳을 보는아름다운 너와 나그림을 그려갔어모래 위 떨린 손끝으로날 향해 웃어주는 입술끝 모양과 저 달보다사랑스러운 두 눈을 가진 네가다정히 내 이름을부르면 내 마음이녹아내려 언제나나 하날 위해 준비된선물 같아 널 안으면잠들지 않는 바다 위를너와 함께 걷는 거 같아하늘 아래 너와 나남은 내 모든 날을 너에게다 줘도 안 아까워이제서야 내가사랑에 눈을 뜬 건 가봐경험하지 못했고믿기지 않는 일만일어나고 있는 거 아니5월에 피는 봄꽃 처럼내 사랑도 피어나부는 바람에 살며시 실어 보내다정히 내 이름을부르면 내 마음이녹아내려 언제나나 하날 위해 준비된선물 같아 널 안으면잠들지 않는 바다 위를너와 함께 걷는 거 같아하늘 아래 너와 나설레는 지금이꿈이 아닌 걸 알잖아난 더 바랄 게 없어먼 훗날 내 생애아주 작은 소리로귓가에 말할 거야내 곁에 머물러줘서마음 다해 고맙다고어떤 말로도 표현 못 할내 인생의 전부였다고이 순간 너를 꼭 안고고백할게 언제나우리 모든 날들과별빛 아래 우리 그림자제일로 특별하다고길고 긴 겨울 끝을 지나너의 앞에 서있는 지금선명한 목소리로널 사랑한다고말할게,잊지는 말아요 함께 했던 날들눈물이 날 때면그대 뒤를 돌아보면 돼요아프지 말아요 쓸쓸한 마음에힘든 일이 참 많았죠 그대에겐내가 곁에 있을게요그댈 위해 노래할게요잊지 말아요그댈 위해 약속할게요언젠가 우리 시간 지나면알게 되겠죠그땐 웃을 수 있죠 나 약속할게요그땐 미소 짓겠죠작은 행복까지 모두외롭고 지칠 때 손잡아 줄게요슬픔이 짙어질 때면 위로해 줄그 한 사람이 될게요그대 부디 울지 말아요슬퍼 말아요그대에게 빛이 될게요언젠가 우리 시간 지나면알게 되겠죠그땐 웃을 수 있죠 나 약속할게요폭풍 속에 혼자 남아 헤매도 오길이 되어 지킬게요 그대그댈 위해 노래할게요잊지 말아요그댈 위해 약속할게요어두운 길을 밝게 비추는그대의 빛이 될게요,둘이 손잡고 걸을 때마주 보며 또 웃을 때사랑한다 말하지 않아도 알아널 만나러 가는 길에내 입가에 늘 환한 미소만이흐르고 있어사랑해 너를 사랑해눈물 나도록 사랑해저기 밤하늘의 별보다보석보다 네가 더 소중해얼마나 내가 얼마나 너를 사랑하는지내 모든 걸 너에게 줄게속상하고 힘이 들 때열이 나고 아파할 때옆에서 늘 힘이 돼준 사람 너야집에 돌아오는 길에내 귓가엔 늘 너의 음성만이흐르고 있어사랑해 너를 사랑해눈물 나도록 사랑해저기 밤하늘의 별보다보석보다 네가 더 소중해얼마나,너를 위해 해 줄 것이 하나 없어서보낼 수밖에 없었고네가 없이 사는 법을 알지 못해서순간순간을 울었다후회로 가득한 한숨 자락이시린 바람처럼가슴 치는 날 그 언젠가우리 만날 수 있을까다시 만날 수 있을까그리 좋던 예전처럼 그때처럼되돌아갈 수 있을까다시 우리가 만나면무엇을 해야만 할까서로를 품에 안고서하염없이 눈물만 흘려볼까붙잡을 마음이야 없었겠냐마는그때 난 부끄러웠다떳떳하게 일어나 널 다시 찾아갈뜨거운 꿈만 꾸었다둘이 함께 했던 순간순간이시린 폭포처럼쏟아지는 날 그 언젠가우리 만날 수 있을까다시 만날 수 있을까그리 좋던 예전처럼 그때처럼되돌아갈 수 있을까다시 우리가 만나면무엇을 해야만 할까서로를 품에 안고서하염없이 눈물만 흘려볼까그리운 마음이 서럽게 흘러넘쳐너에게 닿을 때우리 만날 수 있을까다시 만날 수 있을까그리 좋던 예전처럼 그때처럼되돌아갈 수 있을까다시 우리가 만나면무엇을 해야만 할까서로를 품에 안고서하염없이 눈물만 흘려볼까,나도 한때는 그이의 손을 잡고내가 온 세상 주인공이 된 듯꽃송이의 꽃잎 하나하나까지모두 날 위해 피어났지올림픽대로 뚝섬 유원지서촌 골목골목 예쁜 식당나를 휘청거리게 만든주옥같은 대사들다시 누군가 사랑할 수 있을까예쁘다는 말 들을 수 있을까하루 단 하루만 기회가 온다면죽을힘을 다해 빛나리언제부턴

In [85]:
song_simi_lyric.mean()

0.05035397159343402

In [47]:
lyric_result.iloc[:,0].sort_values(ascending=False).head(20)

Lyric
너와 함께 하고 싶은 일들을상상하는 게요즘 내 일상이 되고너의 즐거워하는 모습을 보고 있으면자연스레 따라 웃고 있는 걸너의 행동에 설레어하고 뒤척이다가지새운 밤이 많아지는데이건 누가 봐도 사랑일 텐데종일 함께면 질릴 텐데나 돌아서도 온통 너인 건아무래도 사랑인가 봐점점 너와 하고 싶은 일들 생각하면서하룰 보낸 날이 많아지는데이건 누가 봐도 사랑일 텐데종일 함께면 질릴 텐데나 돌아서도 온통 너인 건아무래도 사랑인가 봐너의 행복해하는 모습을 보고 있으면나도 모르게 따라 웃는데이 정도면 알아줄 만하잖아너도 용기 낼만 하잖아나만 이런 게 아니라면우리 만나볼 만하잖아아무래도 사랑인가 봐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [71]:
song_simi_co = (
                 + song_simi_genre * 0.3 # 공식 1. 카테고리 유사도
                 + song_simi_artist * 0.2 # 공식 2. 메뉴 유사도
                 + song_simi_lyric * 1 # 공식 3. 리뷰 유사도
                 )

In [72]:
song_simi_co_sorted_ind = song_simi_co.argsort()[:, ::-1] 
song_simi_co_sorted_ind

array([[   0,   63,  328, ..., 1556, 1558, 3573],
       [   1,   16,  331, ..., 1445, 1446, 3573],
       [   2,  113,   51, ..., 1495, 1496, 1486],
       ...,
       [3571, 3536, 3407, ..., 1638, 1639, 1385],
       [3572, 3557, 3416, ..., 1563, 1565, 1786],
       [3573, 3486, 3548, ..., 2033, 2032,    0]], dtype=int64)

In [73]:
pd.DataFrame(song_simi_co_sorted_ind).head()

,0,1,2,3,4,5,6,7,8,9,...,3564,3565,3566,3567,3568,3569,3570,3571,3572,3573
0,0,63,328,51,113,497,153,10,375,137,...,1538,1540,1541,3079,1547,1551,3077,1556,1558,3573
1,1,16,331,362,476,467,180,466,129,8,...,1426,1428,1431,1433,1434,1437,1444,1445,1446,3573
2,2,113,51,1577,296,189,176,379,400,272,...,1481,2972,1487,1488,1489,1492,1493,1495,1496,1486
3,3,6,69,31,27,8,341,20,460,153,...,1631,1632,2949,1636,1637,2948,1639,2947,1642,3573
4,4,14,218,97,495,186,99,35,319,64,...,1191,1187,1184,1179,1176,1174,1170,1168,1166,3573


In [74]:
song_simi_co = (
                 + song_simi_genre * 0.3 # 공식 1. 장르 유사도
                 + song_simi_artist * 0.2 # 공식 2. 가수 유사도
                 + song_simi_lyric * 1 # 공식 3. 가사 유사도
                 )

song_simi_co_sorted_ind = song_simi_co.argsort()[:, ::-1] 


# 최종 구현 함수
def find_simi_place(df, sorted_ind, title, top_n=10):
    
    place_title = df[df['Title'] == title]                    # 입력된 데이터프레임에서 입력된 가사 행을 추출하여 데이터 프레임 생성
    place_index = place_title.index.values                                #생성된 데이터 프레임의 인덱스 값 추출
    similar_indexes = sorted_ind[place_index, :(top_n)]                #입력된 인덱스 어레이에서 이전에 추출한 인덱스 행만 추출
    similar_indexes = similar_indexes.reshape(-1)                         # 2차원 행렬을 1차원으로 변경
    return df.iloc[similar_indexes]


# 하랑을 포함해 5개 업체를 뽑아봅시다.
find_simi_place(song, song_simi_co_sorted_ind, '사랑인가 봐', 5)

,index,Genre,Artist,Title,Lyric,Date,Emotion,Proba
0,0,"발라드, 국내드라마",멜로망스,사랑인가 봐,너와 함께 하고 싶은 일들을상상하는 게요즘 내 일상이 되고너의 즐거워하는 모습을 보...,2022.02.18,슬픔,64.12
63,63,"발라드, 국내드라마",김세정,사랑인가 봐 (Acoustic Ver.),너와 함께 하고 싶은 일들을상상하는 게요즘 내 일상이 되고너의 즐거워하는 모습을 보...,2022.04.05,슬픔,64.12
328,328,"발라드, 국내드라마",멜로망스,우리의 이야기,홀로 집에 돌아오던 나날들은지친 맘을 안아주는 날이 됐고외로움 가득 차 있던 텅 빈...,2021.10.15,슬픔,77.80
51,51,발라드,멜로망스,그게 더 편할 것 같아 (N번째 연애 X 멜로망스),너를 위해 해줬었던 말들은부담을 줬고네가 내게 해줬던 위로에나 상처 입었죠사랑이란 ...,2021.11.26,슬픔,82.43
113,113,발라드,멜로망스,You,어떻게 난 해야 해 니가 떠난다고 말하면 난 혼자 남겨지는 게 더 두려울 텐데 이유...,2018.01.22,슬픔,75.56


In [76]:
song[song["Emotion"]=="놀람"].head()

,index,Genre,Artist,Title,Lyric,Date,Emotion,Proba
228,228,"발라드, 인디음악",CHEEZE (치즈),Madeleine Love,오늘같이 싱그러운 날길거리 차도 별로 다니지 않아 문득 지나가다 거울을 보면 오늘 ...,2015.05.13,놀람,34.54
504,504,댄스,IVE (아이브),ELEVEN,따분한 나의 눈빛이무표정했던 얼굴이널 보며 빛나고 있어널 담은 눈동자는 odd내 안...,2021.12.01,놀람,33.28
510,510,댄스,싸이 (PSY),Celeb,You so popular be ma formula May I call you a ...,2022.04.29,놀람,40.24
521,521,댄스,오마이걸 (OH MY GIRL),Real Love,우린 이 음악을 빌려이 분위기를 빌려지금 이 공기 이건 마치7시간 비행 뒤 만난 것...,2022.03.28,놀람,55.27
540,540,댄스,Kep1er (케플러),WA DA DA,HeyIt’s you& ILet’s startN N Now 도착한 이 PlaceWa...,2022.01.03,놀람,28.15


In [77]:
#고담식당을 포함해 5개 업체를 뽑아봅시다.
find_simi_place(song, song_simi_co_sorted_ind, 'Madeleine Love', 5)

,index,Genre,Artist,Title,Lyric,Date,Emotion,Proba
228,228,"발라드, 인디음악",CHEEZE (치즈),Madeleine Love,오늘같이 싱그러운 날길거리 차도 별로 다니지 않아 문득 지나가다 거울을 보면 오늘 ...,2015.05.13,놀람,34.54
1733,1904,"발라드, 인디음악",CHEEZE (치즈),Romance,빨간 풍선에 가득 담긴꽃잎에 짙은 향기 품고서무표정으로 내게 다가와귓가에 깊이 색을...,2015.05.13,슬픔,40.69
1543,1668,"발라드, 인디음악",CHEEZE (치즈),좋아해 (bye),문득 생각났어너의 그 웃음이익숙했던 너의 그 향기가언제부터인지낯설게 느껴져마음이 붕...,2017.02.20,슬픔,59.15
1635,1778,"발라드, 인디음악",CHEEZE (치즈),Be There,눈이 감기는 나른한 오후 불어오는 봄바람에 문득 스쳐 지나간 그대 생각 지금쯤 뭐할...,2017.05.18,행복,40.98
148,148,"발라드, 인디음악",CHEEZE (치즈),이렇게 좋아해 본 적이 없어요,따스한 바람이 불어오는 날엔포근한 미소로 나를 또 반겨주던그대란 사람 설레는 이 맘...,2021.03.14,슬픔,62.11


## 같은 artist가 대부분 나오는 상황 ->  artist 빼고 진행

In [78]:
song_simi_co = (
                 + song_simi_genre * 0.3 # 공식 1. 카테고리 유사도
#                  + song_simi_artist * 0.2 # 공식 2. 메뉴 유사도
                 + song_simi_lyric * 1 # 공식 3. 리뷰 유사도
                 )

song_simi_co_sorted_ind = song_simi_co.argsort()[:, ::-1] 


# 최종 구현 함수
def find_simi_place(df, sorted_ind, title, top_n=10):
    
    place_title = df[df['Title'] == title]                    # 입력된 데이터프레임에서 입력된 가게이름 행을 추출하여 데이터 프레임 생성
    place_index = place_title.index.values                                #생성된 데이터 프레임의 인덱스 값 추출
    similar_indexes = sorted_ind[place_index, :(top_n)]                #입력된 인덱스 어레이에서 이전에 추출한 인덱스 행만 추출
    similar_indexes = similar_indexes.reshape(-1)                         # 2차원 행렬을 1차원으로 변경
    return df.iloc[similar_indexes]


# 하랑을 포함해 5개 업체를 뽑아봅시다.
find_simi_place(song, song_simi_co_sorted_ind, '사랑인가 봐', 5)

,index,Genre,Artist,Title,Lyric,Date,Emotion,Proba
0,0,"발라드, 국내드라마",멜로망스,사랑인가 봐,너와 함께 하고 싶은 일들을상상하는 게요즘 내 일상이 되고너의 즐거워하는 모습을 보...,2022.02.18,슬픔,64.12
63,63,"발라드, 국내드라마",김세정,사랑인가 봐 (Acoustic Ver.),너와 함께 하고 싶은 일들을상상하는 게요즘 내 일상이 되고너의 즐거워하는 모습을 보...,2022.04.05,슬픔,64.12
497,497,"발라드, 국내드라마",이선희,그대 손 놓아요,어지러이 물든 그대란 세상 속달이 저문 밤 하늘 같죠스치는 바람에 꽃잎들조차하나둘씩...,2022.01.01,슬픔,75.84
153,153,"발라드, 국내드라마",어쿠스틱 콜라보,"묘해, 너와",니 생각에 꽤 즐겁고 니 생각에 퍽 외로워 이상한 일이야 누굴 좋아한단 건 아무 ...,2014.09.02,슬픔,66.00
10,10,"발라드, 국내드라마",성시경,너의 모든 순간,이윽고 내가 한눈에너를 알아봤을 때모든 건 분명 달라지고 있었어내 세상은 널 알기 ...,2014.02.12,슬픔,59.44


In [80]:
find_simi_place(song, song_simi_co_sorted_ind, 'Madeleine Love', 5)

,index,Genre,Artist,Title,Lyric,Date,Emotion,Proba
228,228,"발라드, 인디음악",CHEEZE (치즈),Madeleine Love,오늘같이 싱그러운 날길거리 차도 별로 다니지 않아 문득 지나가다 거울을 보면 오늘 ...,2015.05.13,놀람,34.54
1697,1859,"발라드, 인디음악",스탠딩 에그,사랑하는 너에게 (취향저격 그녀 X 스탠딩 에그),날 보는 두 눈은사랑으로 가득 차 있어그대 손을 잡고오늘도 난 웃어요널 보는 내 눈...,2020.10.16,슬픔,43.94
1649,1803,"발라드, 인디음악",하은요셉,여자친구,나의 여자친구가 되어주겠니이런 내 마음을 넌 받아주겠니넌 누구보다 예뻐넌 뭘 해도 ...,2019.03.07,분노,36.54
1654,1809,"발라드, 인디음악",정동원,이별하기 좋은 날씨,올해 봄은 꽃도 빨리 지고하늘이 높아이렇게 파란 하늘을 보며우리 이별해야 해항상 온...,2019.04.20,슬픔,48.08
1778,1954,"발라드, 인디음악",디에이드 (The Ade),"묘해, 너와 (re:cord)",니 생각에 꽤 즐겁고니 생각에 퍽 외로워이상한 일이야 누굴 좋아한단 건아무 일도 없...,2020.06.25,슬픔,74.85
